# 대전광역시 흡연위험지수 산출 및 시각화

지자체에서 제공하는 공공데이터를 토대로 흡연 및 금연지역과 관련된 데이터가 포함되어있습니다.
EDA의 목표는 흡연지수산출을 위한 각 feature들의 패턴 및 인사이트 발견입니다.

## 데이터셋 살펴보기

총 7개의 csv 파일을 로드합니다.

1. 과태료 (2807, 7)
2. 금연구역 (44475, 7)
3. 병의원 (2300, 7)
4. 주차장 (850, 6)
5. 상권정보 (75145, 9)
6. 지역사회건강조사 설문 (4610, 6)
7. 주민등록인구 (164, 9)

## 라이브러리 및 함수 준비

In [51]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 100, "display.max_columns", 50)
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

color = sns.color_palette()

In [5]:
PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data/전처리 완료 데이터/'
# PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data/전처리 완료 데이터/'


In [ ]:
# colab용 구글 드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

## 데이터 로드

In [55]:
fine = pd.read_csv(PATH + '과태료_전처리(완)_1205.csv', encoding='cp949')
nszone = pd.read_csv(PATH + '금연구역_전처리(완)_1205.csv', encoding='cp949')
hospital = pd.read_csv(PATH + '병의원_전처리(완)_1205.csv', encoding='cp949')
parking = pd.read_csv(PATH + '주차장_전처리(완)_1205.csv', encoding='cp949')
comm = pd.read_csv(PATH + '상권정보_전처리(완)_1205.csv', encoding='cp949')
survey = pd.read_csv(PATH + '지역사회건강조사_전처리(완)_1205.csv', encoding='cp949')
population = pd.read_csv(PATH + '주민등록인구_전처리(완)_1205.csv', encoding='cp949')

print('done')

done


In [20]:
fine.shape, nszone.shape, hospital.shape, parking.shape, comm.shape, survey.shape, population.shape

((2807, 7), (44475, 7), (2300, 7), (850, 6), (75145, 9), (4610, 6), (164, 9))

### 데이터 병합

In [56]:
# df에 모든 컬럼을 추가함
df = pd.concat([fine, nszone], ignore_index=True)
df = pd.concat([df, hospital], ignore_index=True)
df = pd.concat([df, parking], ignore_index=True)
df = pd.concat([df, comm], ignore_index=True)
df = pd.concat([df, survey], ignore_index=True)
df = pd.concat([df, population], ignore_index=True)

#구, 행정동 기준으로 정렬
df.sort_values(by=['구', '행정동'], inplace=True, ignore_index=True)

df

,구분,구,행정동,경도,위도,부과일자,최초본세,분류,이름,병원유형,주차장유형,대분류,중분류,소분류,성별,담배흡연,액상흡연,total,0~19세,20~39세,40~59세,60 over
0,설문,대덕구,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,남자,1.0,8.0,NaN,NaN,NaN,NaN,NaN
1,설문,대덕구,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,여자,8.0,8.0,NaN,NaN,NaN,NaN,NaN
2,설문,대덕구,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,여자,8.0,8.0,NaN,NaN,NaN,NaN,NaN
3,설문,대덕구,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,남자,1.0,8.0,NaN,NaN,NaN,NaN,NaN
4,설문,대덕구,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,남자,8.0,8.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130346,금연구역,중구,필동,126.992913,37.562156,NaN,NaN,담배소매인업소,롤로코리아(대흥점),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130347,금연구역,중구,필동,126.993242,37.562527,NaN,NaN,"음식점(휴게, 일반, 제과)",서씨네왕족발,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130348,금연구역,중구,필동,126.992633,37.562248,NaN,NaN,"음식점(휴게, 일반, 제과)",녹두빈대떡식당,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130349,금연구역,중구,필동,126.992916,37.561810,NaN,NaN,"음식점(휴게, 일반, 제과)",강여사식당,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df.isnull().sum()

구분              0
구               0
행정동             0
경도           4774
위도           4774
부과일자       127544
최초본세       127544
분류          85876
이름          10135
병원유형       128051
주차장유형      129501
대분류         55206
중분류         55206
소분류         55206
성별         125577
담배흡연       125741
액상흡연       125741
total      130187
0~19세      130187
20~39세     130187
40~59세     130187
60 over    130187
dtype: int64

데이터프레임 조작을 위해 결측치를 전부 0으로 변경

In [58]:
df.fillna(0, inplace=True)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130351 entries, 0 to 130350
Data columns (total 22 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   구분       130351 non-null  object 
 1   구        130351 non-null  object 
 2   행정동      130351 non-null  object 
 3   경도       130351 non-null  float64
 4   위도       130351 non-null  float64
 5   부과일자     130351 non-null  object 
 6   최초본세     130351 non-null  float64
 7   분류       130351 non-null  object 
 8   이름       130351 non-null  object 
 9   병원유형     130351 non-null  object 
 10  주차장유형    130351 non-null  object 
 11  대분류      130351 non-null  object 
 12  중분류      130351 non-null  object 
 13  소분류      130351 non-null  object 
 14  성별       130351 non-null  object 
 15  담배흡연     130351 non-null  float64
 16  액상흡연     130351 non-null  float64
 17  total    130351 non-null  float64
 18  0~19세    130351 non-null  float64
 19  20~39세   130351 non-null  float64
 20  40~59세   130351 non-null  

In [62]:
# 메모리 관리를 위한 데이터타입 지정
df = df.astype({'구분':'category',
             '구':'category',
             '행정동':'category',
             '경도' : np.float32,
             '위도' : np.float32,
             '최초본세' : np.uint32,
             '성별' : 'category',
             '담배흡연' : np.uint8, # 소수점 제거 후 카테고리로 변경
             '액상흡연' : np.uint8, # 소수점 제거 후 카테고리로 변경
             'total' : np.uint32,
             '0~19세' : np.uint32,
             '20~39세' : np.uint32,
             '40~59세' : np.uint32,
             '60 over' : np.uint16,
             })

df = df.astype({'담배흡연':'category', '액상흡연':'category'})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130351 entries, 0 to 130350
Data columns (total 22 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   구분       130351 non-null  category
 1   구        130351 non-null  category
 2   행정동      130351 non-null  category
 3   경도       130351 non-null  float32 
 4   위도       130351 non-null  float32 
 5   부과일자     130351 non-null  object  
 6   최초본세     130351 non-null  uint32  
 7   분류       130351 non-null  object  
 8   이름       130351 non-null  object  
 9   병원유형     130351 non-null  object  
 10  주차장유형    130351 non-null  object  
 11  대분류      130351 non-null  object  
 12  중분류      130351 non-null  object  
 13  소분류      130351 non-null  object  
 14  성별       130351 non-null  category
 15  담배흡연     130351 non-null  category
 16  액상흡연     130351 non-null  category
 17  total    130351 non-null  uint32  
 18  0~19세    130351 non-null  uint32  
 19  20~39세   130351 non-null  uint32  
 20  40~5

In [63]:
df.head(2)

,구분,구,행정동,경도,위도,부과일자,최초본세,분류,이름,병원유형,주차장유형,대분류,중분류,소분류,성별,담배흡연,액상흡연,total,0~19세,20~39세,40~59세,60 over
0,설문,대덕구,1,0.0,0.0,0,0,0,0,0,0,0,0,0,남자,1,8,0,0,0,0,0
1,설문,대덕구,1,0.0,0.0,0,0,0,0,0,0,0,0,0,여자,8,8,0,0,0,0,0


In [64]:
# df 저장
df.to_pickle(PATH + 'EDA.pkl')

In [65]:
# df 불러오기
df = pd.read_pickle(PATH + 'EDA.pkl')

## 과태료

체크리스트

* 과태료의 형태 
* 연도별 과태료 발생 추이
* 구 별 과태료 발생 현황
    * 액수 합계
    * 발생 건수 합계
* 행정동 별 과태료 발생 현황
     * 액수 합계
     * 발생 건수 합계
* 과태료의 형태

In [66]:
fine = df[df['구분']=='과태료']
fine.head(2)

,구분,구,행정동,경도,위도,부과일자,최초본세,분류,이름,병원유형,주차장유형,대분류,중분류,소분류,성별,담배흡연,액상흡연,total,0~19세,20~39세,40~59세,60 over
917,과태료,대덕구,대화동,127.416588,36.363914,2016-10-04,80000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
918,과태료,대덕구,대화동,127.415047,36.363464,2017-11-27,80000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
# null check
fine.isnull().sum()

구분         0
구          0
행정동        0
경도         0
위도         0
부과일자       0
최초본세       0
분류         0
이름         0
병원유형       0
주차장유형      0
대분류        0
중분류        0
소분류        0
성별         0
담배흡연       0
액상흡연       0
total      0
0~19세      0
20~39세     0
40~59세     0
60 over    0
dtype: int64

## 금연구역

## 상권정보

## 인구

## 지역사회건강조사(흡연율)